In [1]:
#import libraries
import pandas as pd
pd.set_option("display.max_columns", None)
import calendar
pd.set_option('display.float_format', lambda x: '%.1f' % x)

In [2]:
#print csv
df=pd.read_csv("owid-covid-data.csv")

In [3]:
#number of unique location records
df['location'].nunique()

244

In [4]:
#print unique location records
df['location'].unique()

array(['Afghanistan', 'Africa', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Asia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire Sint Eustatius and Saba',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Cayman Islands', 'Central African Republic', 'Chad', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo', 'Cook Islands',
       'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Curacao',
       'Cyprus', 'Czechia', 'Democratic Republic of Congo', 'Denmark',
       'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethi

In [5]:
#print unique continent records
df['continent'].unique()

array(['Asia', nan, 'Europe', 'Africa', 'North America', 'South America',
       'Oceania'], dtype=object)

In [6]:
#remove nan from continent
df = df[df["continent"].str.contains("nan") == False]

In [7]:
df['location'].nunique()

231

In [8]:
#df info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180022 entries, 0 to 191134
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    180022 non-null  object 
 1   continent                                   180022 non-null  object 
 2   location                                    180022 non-null  object 
 3   date                                        180022 non-null  object 
 4   total_cases                                 172499 non-null  float64
 5   new_cases                                   172293 non-null  float64
 6   new_cases_smoothed                          171184 non-null  float64
 7   total_deaths                                154245 non-null  float64
 8   new_deaths                                  154053 non-null  float64
 9   new_deaths_smoothed                         152955 non-null  float64
 

In [9]:
#drop columns 
df = df[df.columns.drop(list(df.filter(regex='smoothed')))]
df = df[df.columns.drop(list(df.filter(regex='per_million')))]
df = df[df.columns.drop(list(df.filter(regex='mortality')))]
df = df[df.columns.drop(list(df.filter(regex='per_hundred')))]
df = df[df.columns.drop(list(df.filter(regex='tests')))]
df = df[df.columns.drop(list(df.filter(regex='weekly')))]
df.drop(['cardiovasc_death_rate','iso_code','diabetes_prevalence',
         'gdp_per_capita','reproduction_rate','aged_70_older','extreme_poverty',
        'aged_65_older','female_smokers','male_smokers'], axis=1,inplace=True)

In [10]:
df.columns

Index(['continent', 'location', 'date', 'total_cases', 'new_cases',
       'total_deaths', 'new_deaths', 'icu_patients', 'hosp_patients',
       'positive_rate', 'total_vaccinations', 'people_vaccinated',
       'people_fully_vaccinated', 'total_boosters', 'new_vaccinations',
       'stringency_index', 'population', 'population_density', 'median_age',
       'handwashing_facilities', 'hospital_beds_per_thousand',
       'life_expectancy', 'human_development_index'],
      dtype='object')

In [11]:
#to datetime
df['date'] =  pd.to_datetime(df['date'])

In [12]:
df['date']

0        2020-02-24
1        2020-02-25
2        2020-02-26
3        2020-02-27
4        2020-02-28
            ...    
191130   2022-05-29
191131   2022-05-30
191132   2022-05-31
191133   2022-06-01
191134   2022-06-02
Name: date, Length: 180022, dtype: datetime64[ns]

In [13]:
#split the date to day year month
df['day'] = df['date'].dt.day
df['month_num'] = df['date'].dt.month
df['year'] = df['date'].dt.year

In [14]:
#print those columns
df[["day", "month_num","year"]]

,day,month_num,year
0,24,2,2020
1,25,2,2020
2,26,2,2020
3,27,2,2020
4,28,2,2020
...,...,...,...
191130,29,5,2022
191131,30,5,2022
191132,31,5,2022
191133,1,6,2022


In [15]:
#getting month names 
df['month_name'] = df['month_num'].apply(lambda x: calendar.month_abbr[x])

In [16]:
df['month_name']

0         Feb
1         Feb
2         Feb
3         Feb
4         Feb
         ... 
191130    May
191131    May
191132    May
191133    Jun
191134    Jun
Name: month_name, Length: 180022, dtype: object

In [17]:
#save csv
df.to_csv('df_cleaned.csv')

In [18]:
#cases population df for report script
test = df.groupby('location')['new_cases'].sum()
test=pd.DataFrame(test)
test.reset_index(inplace=True)
test.rename(columns = {'new_cases':'total_cases'}, inplace = True)
test2=df.groupby('location')['population'].max()
test2=pd.DataFrame(test2)
test2.reset_index(inplace=True)
cases_population=pd.merge(test ,test2, on='location')
cases_population.to_csv('cases_population.csv')